In [108]:
import pygame_2d
from const import *
import numpy as np

screen = np.zeros((720, 1280, 3), dtype=np.uint8)


gamma = 0.99
alpha = 0.1
epsilon = 1
epsilon_decay = epsilon / 4000
n_epsilondes = 10000


In [109]:


def pick_sample(state):
    if np.random.random() > epsilon:
        action = np.argmax(q_table[tuple(state)])
    else:
        action = np.random.randint(0, ACTION_SPACE)
    return action


game = pygame_2d.PyGame2D(screen=screen)

lidarspace_shape = tuple([LENGTH_LIDARSIGNAL] * SECTIONS_LIDARSPACE)
new_observation_shape = (ALPHA_SPACE, FWVELO_SPACE,
                         RVELO_SPACE, SECTIONS_LIDARSPACE) + lidarspace_shape

new_observation_shape

(20, 4, 4, 4, 3, 3, 3, 3)

In [110]:
q_table = np.zeros(new_observation_shape + (ACTION_SPACE,))
q_table.shape

(20, 4, 4, 4, 3, 3, 3, 3, 6)

In [111]:
state = game.observe()
state

array([   10,     2,     2,     2,     2,     2, 99999], dtype=int64)

In [112]:

reward_records = []
for i in range(n_epsilondes):
    done = False
    total_reward = 0
    state = game.observe()

    while not done:
        action = pick_sample(state)
        game.action(action)

        next_state = game.observe()
        reward = game.evaluate()
        done = game.is_done()

        # Update Q-Table
        maxQ = np.max(q_table[tuple(next_state)])
        q_table[tuple(state)][action] += alpha * (reward +
                                                  gamma * maxQ - q_table[tuple(state)][action])

        state = next_state
        total_reward += reward

    # Update epsilon for each episode
    if epsilon - epsilon_decay >= 0:
        epsilon -= epsilon_decay

    # Record total rewards in episode
    print("Run episode {} with rewards {}".format(i, total_reward), end="\r")
    reward_records.append(total_reward)

print("\nDone")


0.013117790222167969 0.013117790222167969 0.013117790222167969 0.013117790222167969 1


IndexError: index 99999 is out of bounds for axis 6 with size 3